# Particle Swarm Optimization

### (using functions)

In [15]:
def pso(fitness_func, lbound, ubound, swarm_size=20, max_iter=40, dimensions=4, phip=1.494, phig=1.494, weight=0.5):

In [1]:
# defining the fitness funcion to be optimised by pso
# weights is the list of weights that are to be calculated by PSO
# 'user' is the user for which we want to predict the weights
user = pd.DataFrame()
def fitness_func(weights):
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    w_sum = w1+w2+w3+w4
    w1 = w1/w_sum
    w2 = w2/w_sum
    w3 = w3/w_sum
    w4 = w4/w_sum
    ans = 0
    for index, row in user.iterrows():
        ans += abs((w1*row.criterion1 + w2*row.criterion2 + w3*row.criterion3 + w4*row.criterion4) - row.overall)
    ans /= len(user.movie_id.unique())
#     print(len(user.movie_id.unique()))
#     ans = math.sqrt(ans)
    return ans
# math.sqrt(10)

NameError: name 'pd' is not defined

In [ ]:
# function for predicting the label according to our weights got using pso
def predict_pso(xopt, test):
    predicted_list = list()
    w1 = xopt[0]
    w2 = xopt[1]
    w3 = xopt[2]
    w4 = xopt[3]
    w_sum = w1+w2+w3+w4
    w1 = w1/w_sum
    w2 = w2/w_sum
    w3 = w3/w_sum
    w4 = w4/w_sum
    for index, row in test.iterrows():
        prediction = (w1*row.criterion1 + w2*row.criterion2 + w3*row.criterion3 + w4*row.criterion4)
        predicted_list.append(prediction)
    return predicted_list

In [ ]:
# Anubhav's implementation of PSO
# def pso(fitness_func, lbound, ubound, swarm_size=40, max_iter=70, dimensions=4, phip=1.494, phig=1.494, weight=0.5):

#checking and defining weight inside the function; a random number between 0.5 and 1
def pso(fitness_func, lbound, ubound, swarm_size=40, max_iter=50, dimensions=4, phip=1.494, phig=1.494):
    x=[]            # positon of each particle
    x_pbest=[]      # best position of each particle 
    x_gbest=[]      # best global position of population
    v=[]            # velocity of each particle
    v_min=[]
    v_max=[]
    
    # assign initial random positions to the particles
    for i in range(swarm_size):
        x.append([])
        x_pbest.append([])
        for j in range(dimensions):
            x[i].append(random.uniform(lbound[j],ubound[j]))
            x_pbest[i].append(x[i][j])
            
    #calculate initial group best of the population
    for i in range(swarm_size):
        if i==0:
            x_gbest=x[i]
        elif fitness_func(x[i])<fitness_func(x_gbest):
            x_gbest=x[i]

    
    # calculate minimum and maximum boundaries of velocity vector
    for i in range(dimensions):
        v_min.append(-(ubound[i]-lbound[i])/100)
        v_max.append((ubound[i]-lbound[i])/100)
 
    # assign initial random velocities to the particles
    for i in range(swarm_size):
        v.append([])
        for j in range(dimensions):
            v[i].append(random.uniform(v_min[j],v_max[j]))
            
    for iter in range(max_iter):
        for i in range(swarm_size):
            for j in range(dimensions):
                r1=random.uniform(0,1)
                r2=random.uniform(0,1)
                weight = random.uniform(0.5, 1)
                # calculate new velocity for each particle
                v[i][j] = weight*(v[i][j]) + r1*phip*(x_pbest[i][j]-x[i][j]) + r2*phig*(x_gbest[j]-x[i][j])
                
                if v[i][j] > v_max[j]:
                    v[i][j] = v_max[j]
                
                if v[i][j] < v_min[j]:
                    v[i][j] = v_min[j]


                # Kumud's code
                #checking the limit of velocity, if goes beyond then sack it
#                 if v[i][j] > v_max:
#                     vel[i][j] = (vel_max / abs(vel[i][j])*vel[i][j])
                                 
                # calculate new position for each particle
                x[i][j] = x[i][j] + v[i][j]
                
                if x[i][j] > ubound[j]:
                    x[i][j] = ubound[j]
                    
                if x[i][j] < lbound[j]:
                    x[i][j] = lbound[j]
            
            if fitness_func(x[i]) < fitness_func(x_pbest[i]):
                x_pbest[i] = x[i]
                
            if fitness_func(x[i]) < fitness_func(x_gbest):
                x_gbest = x[i]
                
    return x_gbest,fitness_func(x_gbest)

In [ ]:
# applying the particle-swarm-optimization
# xopt = the optimal input values
# fopt = the optimal objective value
df = list_datasets[0]
user_list = df.groupby(['user_id'])
lower_bound = [0, 0, 0, 0]
upper_bound = [100, 100, 100, 100]
mae = 0
count = 0
for index, item in user_list:
    # split item in train and test data
    train, test = train_test_split(item, test_size = 0.3, random_state = 1601)
    user = train
    # library implemenatation
#     xopt, fopt = pyswarm.pso(fitness_func, lower_bound, upper_bound)
    # Anubhav's implementation
    xopt, fopt = pso(fitness_func, lower_bound, upper_bound)
    print(xopt)
    print(fopt)
    # predict the values on the test dataset
    predicted_list = predict_pso(xopt, test)
#     print("test shape ", test.shape[0])
    # calculate the mean average error
#     print("checking --> ",test.overall.shape[1])
#     print("debug = ",len(predicted_list))
#     print("debug = ", len(test.overall))
    error = metrics.mean_absolute_error(test.overall.tolist(), predicted_list)
    accuracy = metrics.r2_score(test.overall.tolist(), predicted_list)
    print("mae = ",error)
    mae += error
#     count += 1
#     if(count==20):
#         break
print("average of mean average error: ", mae/len(user_list))